In [1]:
# Question 1
import pandas as pd

In [2]:
file_path = 'urdu-sentiment-corpus-v1.tsv'
df = pd.read_csv(file_path, delimiter='\t')
print(df)

                                                 Tweet Class
0    میں نے ایٹم بم بنایا ھے ۔۔۔۔او بھائی ایٹم بمب ...     P
1    چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن...     N
2                             ٹویٹر کا خیال کیسے آیا ؟     O
3    سرچ انجن گوگل کے نائب صدر نے فضا میں ، 130,000...     P
4      ابھی تک اسکی لہریں کبھی کبھی آ جاتی ہیں یار :أْ     P
..                                                 ...   ...
995     اُس آدمی نے اِس سالار کو کافی معقول ٹپ دی ہے ۔     P
996  چچا غالب کی روح سے معذرت کے ساتھہم نے مانا کہ ...     P
997  واہ جناب واہ! اچھی رہی۔ جناب خود کو فرشتہ سمجو...     P
998  اسلام آباد :پی اے ٹی کا دھرنا ختم، صفائی کے کا...     P
999  دنیا نے کس کا راہ وفا میں دیا ہے ساتھتم بھی چل...     P

[1000 rows x 2 columns]


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, GRU, LSTM, Bidirectional, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


# Split data into features and labels
X = df['Tweet']
y = df['Class']

# Binarization
y_binary = np.where(y == 'P', 1, 0)

# Checking if there are mixed or unknown targets in the labels
if np.any((y_binary != 0) & (y_binary != 1)):
    raise ValueError("Labels contain mixed or unknown targets.")

# Checking for missing values in the labels
missing_labels = np.isnan(y_binary)
if np.any(missing_labels):
    raise ValueError("Missing values found in labels.")

y_binary = np.expand_dims(y_binary, axis=-1)

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
max_sequence_length = 100
X = pad_sequences(X, maxlen=max_sequence_length)

# train and test split
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.25, random_state=42)


def create_model(model_type, num_layers, dropout_rate):
    model = Sequential()

    model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=max_sequence_length))


    # Adding layers based on the model
    if model_type == 'RNN':
        for _ in range(num_layers):
            model.add(SimpleRNN(64, return_sequences=True))
    elif model_type == 'GRU':
        for _ in range(num_layers):
            model.add(GRU(64, return_sequences=True))
    elif model_type == 'LSTM':
        for _ in range(num_layers):
            model.add(LSTM(64, return_sequences=True))
    elif model_type == 'BiLSTM':
        for _ in range(num_layers):
            model.add(Bidirectional(LSTM(64, return_sequences=True)))

    # Dropout layer for dropout rate
    model.add(Dropout(dropout_rate))

    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    return model


results = []

# Hyperparameters
num_layers_options = [2, 3]
dropout_rate_options = [0.3, 0.7]


for model_type in ['RNN', 'GRU', 'LSTM', 'BiLSTM']:
    for num_layers in num_layers_options:
        for dropout_rate in dropout_rate_options:
            # Creating model
            model = create_model(model_type, num_layers, dropout_rate)
            model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

            # Training model
            model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)

            # Model Evaluation
            y_pred_prob = model.predict(X_test)
            y_pred_mean = np.mean(y_pred_prob, axis=1)
            y_pred = (y_pred_mean > 0.5).astype(int)

            # Checking shapes of y_test and y_pred
            if y_test.shape != y_pred.shape:
                raise ValueError("Shapes of y_test and y_pred do not match.")
            # Checking for missing values in y_test
            if np.isnan(y_test).any():
                raise ValueError("Missing values found in y_test.")

            # Checking shapes of y_test and y_pred
            if y_test.shape != y_pred.shape:
                raise ValueError("Shapes of y_test and y_pred do not match.")

            # Calculating evaluation metrics
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)

            results.append({
                'Model': model_type,
                'Num Layers': num_layers,
                'Dropout Rate': dropout_rate,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1-score': f1
            })


results_df = pd.DataFrame(results)
print(results_df)







8/8 [==============================] - 2s 60ms/step


C:\Users\anasb\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


8/8 [==============================] - 5s 101ms/step
     Model  Num Layers  Dropout Rate  Accuracy  Precision    Recall  F1-score
0      RNN           2           0.3     0.520   0.818182  0.070866  0.130435
1      RNN           2           0.7     0.516   0.875000  0.055118  0.103704
2      RNN           3           0.3     0.508   0.508065  0.992126  0.672000
3      RNN           3           0.7     0.496   1.000000  0.007874  0.015625
4      GRU           2           0.3     0.500   1.000000  0.015748  0.031008
5      GRU           2           0.7     0.520   0.705882  0.094488  0.166667
6      GRU           3           0.3     0.628   0.628788  0.653543  0.640927
7      GRU           3           0.7     0.492   0.000000  0.000000  0.000000
8     LSTM           2           0.3     0.532   0.613636  0.212598  0.315789
9     LSTM           2           0.7     0.632   0.610063  0.763780  0.678322
10    LSTM           3           0.3     0.564   0.650000  0.307087  0.417112
11    LSTM 

In [ ]:
# Question 2
# Using RNN with 3 layers and 0.3 dropout rate as it has highest f1-score

In [4]:
#Word2vec
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from gensim.models import KeyedVectors
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, Dropout
from keras.preprocessing.sequence import pad_sequences

# Load the pre-trained Word2Vec embeddings
word2vec_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, limit=20000)

# Load the dataset
file_path = 'urdu-sentiment-corpus-v1.tsv'
df = pd.read_csv(file_path, delimiter='\t')

# Split data into features and labels
X = df['Tweet']
y = df['Class']

# Convert labels to binary format (0 for Negative, 1 for Positive)
y_binary = np.where(y == 'P', 1, 0)

# Tokenizing the text data and convert to sequences
max_sequence_length = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Pad sequences to make them of equal length
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)

X_train, X_test, y_train, y_test = train_test_split(X_padded, y_binary, test_size=0.25, random_state=42)

# Creating an embedding matrix using Word2Vec embeddings
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 300))
for word, i in tokenizer.word_index.items():
    if word in word2vec_model:
        embedding_matrix[i] = word2vec_model[word]

# Defining the RNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=300, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
model.add(SimpleRNN(64, return_sequences=False))  # Return sequences only for the last layer
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

# Predict probabilities for the test set
y_pred_prob = model.predict(X_test)

# Threshold for classification
threshold = 0.5
y_pred = (y_pred_prob > threshold).astype(int)

# Calculate precision, recall, and F1-score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print("Evaluation metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

results = []
for threshold in np.arange(0.1, 0.6, 0.1):
    y_pred = (y_pred_prob > threshold).astype(int)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    results.append({'Threshold': threshold, 'Precision': precision, 'Recall': recall, 'F1-score': f1})


results_df = pd.DataFrame(results)

# Print results
print(results_df)

Epoch 1/10
12/12 [==============================] - 5s 84ms/step - loss: 0.6933 - accuracy: 0.5067
Epoch 2/10
12/12 [==============================] - 1s 68ms/step - loss: 0.6918 - accuracy: 0.5293
Epoch 3/10
12/12 [==============================] - 1s 50ms/step - loss: 0.6825 - accuracy: 0.5613
Epoch 4/10
12/12 [==============================] - 1s 67ms/step - loss: 0.6856 - accuracy: 0.5360
Epoch 5/10
12/12 [==============================] - 1s 53ms/step - loss: 0.6818 - accuracy: 0.5387
Epoch 6/10
12/12 [==============================] - 1s 53ms/step - loss: 0.6815 - accuracy: 0.5400
Epoch 7/10
12/12 [==============================] - 1s 49ms/step - loss: 0.6797 - accuracy: 0.5400
Epoch 8/10
12/12 [==============================] - 1s 50ms/step - loss: 0.6803 - accuracy: 0.5400
Epoch 9/10
12/12 [==============================] - 1s 93ms/step - loss: 0.6803 - accuracy: 0.5347
Epoch 10/10
8/8 [==============================] - 0s 20ms/step
Evaluation metrics:
Accuracy: 0.492
Precision

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, Dropout
from keras.preprocessing.sequence import pad_sequences

# Load the GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

glove_embeddings_path = 'glove.6B.300d.txt'  # Adjust the path to your GloVe embeddings file
glove_embeddings = load_glove_embeddings(glove_embeddings_path)

# Load the dataset
file_path = 'urdu-sentiment-corpus-v1.tsv'
df = pd.read_csv(file_path, delimiter='\t')

# Split data into features and labels
X = df['Tweet']
y = df['Class']

# Convert labels to binary format (0 for Negative, 1 for Positive)
y_binary = np.where(y == 'P', 1, 0)

# Tokenizing the text data and convert to sequences
max_sequence_length = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Pad sequences to make them of equal length
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)

X_train, X_test, y_train, y_test = train_test_split(X_padded, y_binary, test_size=0.25, random_state=42)

# Creating an embedding matrix using GloVe embeddings
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 300))
for word, i in tokenizer.word_index.items():
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Defining the RNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=300, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
model.add(SimpleRNN(64, return_sequences=False))  # Return sequences only for the last layer
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compiling the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the model
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

# Predicting probabilities for the test set
y_pred_prob = model.predict(X_test)

# Threshold for classification
threshold = 0.5
y_pred = (y_pred_prob > threshold).astype(int)

# Calculate precision, recall, and F1-score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print("Evaluation metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

results = []
for threshold in np.arange(0.1, 0.6, 0.1):
    y_pred = (y_pred_prob > threshold).astype(int)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    results.append({'Threshold': threshold, 'Precision': precision, 'Recall': recall, 'F1-score': f1})

results_df = pd.DataFrame(results)

# Print results
print(results_df)


Epoch 1/10
12/12 [==============================] - 7s 103ms/step - loss: 0.7110 - accuracy: 0.5213
Epoch 2/10
12/12 [==============================] - 1s 86ms/step - loss: 0.6851 - accuracy: 0.5440
Epoch 3/10
12/12 [==============================] - 1s 90ms/step - loss: 0.6722 - accuracy: 0.5733
Epoch 4/10
12/12 [==============================] - 1s 80ms/step - loss: 0.6673 - accuracy: 0.5733
Epoch 5/10
12/12 [==============================] - 1s 86ms/step - loss: 0.6602 - accuracy: 0.5893
Epoch 6/10
12/12 [==============================] - 1s 93ms/step - loss: 0.6458 - accuracy: 0.5880
Epoch 7/10
12/12 [==============================] - 1s 85ms/step - loss: 0.6337 - accuracy: 0.6107
Epoch 8/10
12/12 [==============================] - 1s 74ms/step - loss: 0.6329 - accuracy: 0.5880
Epoch 9/10
12/12 [==============================] - 2s 136ms/step - loss: 0.6343 - accuracy: 0.6120
Epoch 10/10
8/8 [==============================] - 1s 63ms/step
Evaluation metrics:
Accuracy: 0.492
Precisi

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from gensim.models import KeyedVectors
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, Dropout
from keras.preprocessing.sequence import pad_sequences

# Load the FastText embeddings
fasttext_model = KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec', binary=False, limit=20000)  # Adjust the path

# Load the dataset
file_path = 'urdu-sentiment-corpus-v1.tsv'
df = pd.read_csv(file_path, delimiter='\t')

# Split data into features and labels
X = df['Tweet']
y = df['Class']

# Convert labels to binary format (0 for Negative, 1 for Positive)
y_binary = np.where(y == 'P', 1, 0)

# Tokenizing the text data and convert to sequences
max_sequence_length = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Pad sequences to make them of equal length
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)

X_train, X_test, y_train, y_test = train_test_split(X_padded, y_binary, test_size=0.25, random_state=42)

# Creating an embedding matrix using FastText embeddings
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 300))
for word, i in tokenizer.word_index.items():
    if word in fasttext_model:
        embedding_matrix[i] = fasttext_model[word]

# Defining the RNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=300, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
model.add(SimpleRNN(64, return_sequences=False))  # Return sequences only for the last layer
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compiling the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

# Predicting probabilities for the test set
y_pred_prob = model.predict(X_test)

# Threshold for classification
threshold = 0.5
y_pred = (y_pred_prob > threshold).astype(int)

# Calculate precision, recall, and F1-score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print("Evaluation metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

results = []
for threshold in np.arange(0.1, 0.6, 0.1):
    y_pred = (y_pred_prob > threshold).astype(int)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    results.append({'Threshold': threshold, 'Precision': precision, 'Recall': recall, 'F1-score': f1})


results_df = pd.DataFrame(results)

# Print results
print(results_df)


Epoch 1/10
12/12 [==============================] - 3s 56ms/step - loss: 0.6954 - accuracy: 0.4920
Epoch 2/10
12/12 [==============================] - 1s 58ms/step - loss: 0.6857 - accuracy: 0.5400
Epoch 3/10
12/12 [==============================] - 1s 54ms/step - loss: 0.6779 - accuracy: 0.5520
Epoch 4/10
12/12 [==============================] - 1s 67ms/step - loss: 0.6780 - accuracy: 0.5373
Epoch 5/10
12/12 [==============================] - 1s 68ms/step - loss: 0.6747 - accuracy: 0.5560
Epoch 6/10
12/12 [==============================] - 1s 55ms/step - loss: 0.6716 - accuracy: 0.5627
Epoch 7/10
12/12 [==============================] - 1s 63ms/step - loss: 0.6904 - accuracy: 0.5360
Epoch 8/10
12/12 [==============================] - 1s 56ms/step - loss: 0.6689 - accuracy: 0.5507
Epoch 9/10
12/12 [==============================] - 1s 60ms/step - loss: 0.6645 - accuracy: 0.5600
Epoch 10/10
8/8 [==============================] - 1s 20ms/step
Evaluation metrics:
Accuracy: 0.52
Precision:

In [12]:
!pip install tensorflow_hub

  Using cached tensorflow_hub-0.16.1-py2.py3-none-any.whl (30 kB)
  Using cached tf_keras-2.16.0-py3-none-any.whl (1.7 MB)
  Using cached tensorflow-2.16.1-cp310-cp310-win_amd64.whl (2.1 kB)
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0


In [21]:
!pip install --upgrade --force-reinstall tensorflow


  Using cached tensorflow-2.16.1-cp310-cp310-win_amd64.whl (2.1 kB)
  Using cached tensorflow_intel-2.16.1-cp310-cp310-win_amd64.whl (376.9 MB)


ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



In [23]:
!pip install --upgrade tensorflow keras


In [16]:
!pip install allennlp

  Using cached allennlp-2.10.1-py3-none-any.whl (730 kB)
  Using cached transformers-4.20.1-py3-none-any.whl (4.4 MB)
  Using cached typer-0.9.0-py3-none-any.whl (45 kB)
  Using cached base58-2.1.1-py3-none-any.whl (5.6 kB)
  Using cached filelock-3.7.1-py3-none-any.whl (10 kB)
  Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
  Using cached more_itertools-10.2.0-py3-none-any.whl (57 kB)
  Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl (101 kB)
  Using cached protobuf-3.20.3-cp310-cp310-win_amd64.whl (904 kB)
  Using cached fairscale-0.4.6-py3-none-any.whl
  Using cached torchvision-0.13.1-cp310-cp310-win_amd64.whl (1.1 MB)
  Using cached cached_path-1.1.6-py3-none-any.whl (26 kB)
  Using cached spacy-3.3.3-cp310-cp310-win_amd64.whl (11.7 MB)
  Using cached wandb-0.12.21-py2.py3-none-any.whl (1.8 MB)
  Using cached boto3-1.34.64-py3-none-any.whl (139 kB)
  Using cached google_cloud_storage-2.15.0-py2.py3-none-any.whl (123 kB)
  Using cached catalogue-2.0.10-py3-none-any

In [27]:
!pip install --upgrade tensorflow-hub

In [30]:
!pip install tensorflow-text

  Using cached tensorflow_text-2.10.0-cp310-cp310-win_amd64.whl (5.0 MB)
     ------------------------------------ 455.9/455.9 MB 273.8 kB/s eta 0:00:00


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\anasb\\anaconda3\\lib\\site-packages\\protobuf-3.20.3.dist-info\\METADATA'



  Using cached tensorflow_text-2.10.0-cp310-cp310-win_amd64.whl (5.0 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.16.1 requires keras>=3.0.0, but you have keras 2.10.0 which is incompatible.
tensorflow-intel 2.16.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
tensorflow-intel 2.16.1 requires tensorboard<2.17,>=2.16, but you have tensorboard 2.10.1 which is incompatible.
tensorboardx 2.6.2.2 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.


     ---------------------------------------- 5.0/5.0 MB 1.7 MB/s eta 0:00:00
     ------------------------------------- 455.9/455.9 MB 37.7 kB/s eta 0:00:00
     ------------------------------------ 895.7/895.7 kB 244.3 kB/s eta 0:00:00
     ---------------------------------------- 5.9/5.9 MB 393.7 kB/s eta 0:00:00
     --------------------------------------- 42.6/42.6 kB 76.6 kB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 255.7 kB/s eta 0:00:00
     ------------------------------------ 438.7/438.7 kB 162.2 kB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 130.5 kB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 326.6 kB/s eta 0:00:00
     ------------------------------------ 781.3/781.3 kB 822.1 kB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.0.5
    Uninstalling keras-3.0.5:
      Successfully uninstalled keras-3.0.5
  Attempting uninstall: tf-keras
    Found existing in

In [34]:
!pip install --upgrade tensorflow tensorflow-text

  Using cached tensorflow-2.16.1-cp310-cp310-win_amd64.whl (2.1 kB)
  Using cached tensorboard-2.16.2-py3-none-any.whl (5.5 MB)
  Using cached keras-3.0.5-py3-none-any.whl (1.0 MB)
  Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl (413 kB)
  Using cached tensorflow-2.10.1-cp310-cp310-win_amd64.whl (455.9 MB)


ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_text as text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, Dropout
#from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
file_path = 'urdu-sentiment-corpus-v1.tsv'
df = pd.read_csv(file_path, delimiter='\t')

# Split data into features and labels
X = df['Tweet']
y = df['Class']

# Convert labels to binary format (0 for Negative, 1 for Positive)
y_binary = np.where(y == 'P', 1, 0)

# Tokenizing the text data and convert to sequences
max_sequence_length = 100
tokenizer = text.keras_tokenizer_from_dataset(X)
X_sequences = tokenizer(X)

# Pad sequences to make them of equal length
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)

X_train, X_test, y_train, y_test = train_test_split(X_padded, y_binary, test_size=0.25, random_state=42)

# Defining the RNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.get_vocabulary()), output_dim=300, input_length=max_sequence_length))
model.add(SimpleRNN(64, return_sequences=True))  # Return sequences for all layers
model.add(Dropout(0.3))
model.add(SimpleRNN(64, return_sequences=True))  # Return sequences for all layers
model.add(Dropout(0.3))
model.add(SimpleRNN(64, return_sequences=False))  # Return sequences only for the last layer
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compiling the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the model
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

# Predicting probabilities for the test set
y_pred_prob = model.predict(X_test)

# Threshold for classification
threshold = 0.5
y_pred = (y_pred_prob > threshold).astype(int)

# Calculate precision, recall, and F1-score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print("Evaluation metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


AttributeError: module 'tensorflow_text' has no attribute 'keras_tokenizer_from_dataset'